In [1]:
from fractions import Fraction
from typing import Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [2]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present (though your files use decimals).
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)

In [3]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

# Phương pháp lũy thừa - tìm giá trị riêng trội nhất

## Thuật toán

### TH1. Trị riêng trội thực, bội đơn hoặc bội đa

In [4]:
def power_method(A,
                 x0,
                 tol=1e-6,
                 max_iter=1000,
                 norm_ord=np.inf,
                 precision=12):
    """
    Power Method for finding the dominant eigenvalue and eigenvector of A.
    Displays each iteration in a pandas DataFrame using the specified norm.

    Parameters:
      A (ndarray): Square matrix (n×n).
      x0 (ndarray): Initial guess vector (n,).
      tol (float): Convergence tolerance for eigenvalue change.
      max_iter (int): Maximum number of iterations.
      norm_ord (int, float, or str): Norm order for both eigenvalue estimation and vector normalization (e.g. 1, 2, np.inf).
      precision (int): Number of decimal places to display.

    Returns:
      eigenvalue (float): Approximate dominant eigenvalue.
      eigenvector (ndarray): Approximate eigenvector corresponding to the dominant eigenvalue.
    """
    # Normalize initial guess
    x = x0 / np.linalg.norm(x0, ord=norm_ord)
    eigenvalue = 0.0
    history = []
    n = A.shape[0]

    for k in range(1, max_iter+1):
        # Compute A x_{k-1}
        y = A.dot(x)
        # Estimate eigenvalue via the chosen norm
        lambda_new = np.linalg.norm(y, ord=norm_ord)
        if lambda_new == 0:
            raise ZeroDivisionError("Norm of A*x is zero; unable to continue.")
        # Normalize to get next eigenvector estimate
        x_new = y / lambda_new

        # Record iteration data: [iter, y1..yn, lambda, x1..xn]
        row = [k] + y.tolist() + [lambda_new] + x_new.tolist()
        history.append(row)

        # Check convergence on eigenvalue
        if abs(lambda_new - eigenvalue) < tol:
            break

        x = x_new
        eigenvalue = lambda_new

    # Build and display DataFrame
    cols = ['Iteration'] + [f'y{i+1}' for i in range(n)] + ['lambda'] + [f'x{i+1}' for i in range(n)]
    df = pd.DataFrame(history, columns=cols)
    print(df.to_string(
        index=False,
        float_format=lambda v: f"{v:.{precision}f}"
    ))

    return eigenvalue, x_new


### TH2. 2 trị riêng trội thực, có trị tuyệt đối bằng nhau

In [5]:
def power_method_case2(
    A,
    x0,
    tol=1e-6,
    max_iter=1000,
    norm_ord=np.inf,
    precision=12
):
    """
    Power Method for when the two largest eigenvalues satisfy |λ1| = |λ2| > |λ3| and λ1 = -λ2.
    Uses even powers of A to separate eigencomponents, printing y and x in one table.

    Parameters:
      A (ndarray): Square matrix (n×n).
      x0 (ndarray): Initial guess vector (n,).
      tol (float): Convergence tolerance for λ^2 change.
      max_iter (int): Maximum number of iterations for k.
      norm_ord (int, float, or str): Norm order for vector normalization.
      precision (int): Decimal places for display.

    Returns:
      lambda1 (float): Approximate dominant eigenvalue (positive).
      x1 (ndarray): Eigenvector corresponding to λ1.
      x2 (ndarray): Eigenvector corresponding to λ2 = -λ1.
    """
    n = A.shape[0]
    A2 = A.dot(A)
    y_even = np.array(x0, dtype=float)
    eigen2 = 0.0
    history = []

    for k in range(1, max_iter + 1):
        if k > 1:
            y_even = A2.dot(y_even)
        # Compute next even power
        y_even2 = A2.dot(y_even)
        # Index of largest magnitude in y_even
        j = np.argmax(np.abs(y_even))
        # Estimate λ^2
        lambda2_new = y_even2[j] / y_even[j]
        # Compute λ
        lambda1 = np.sqrt(lambda2_new)
        # Compute y_{2k+1}
        y_odd = A.dot(y_even)
        # Form eigenvectors
        x1 = y_even + lambda1 * y_odd
        x2 = y_even - lambda1 * y_odd
        # Normalize
        x1 = x1 / np.linalg.norm(x1, ord=norm_ord)
        x2 = x2 / np.linalg.norm(x2, ord=norm_ord)

        # Record iteration: [k, y_even1..n, lambda2, x1_1..n, x2_1..n]
        row = [k] + y_even.tolist() + [lambda2_new] + x1.tolist() + x2.tolist()
        history.append(row)

        # Convergence check on λ^2
        if k > 1 and abs(lambda2_new - eigen2) < tol:
            break
        eigen2 = lambda2_new

    # Display results in pandas
    cols = ['Iteration'] + [f'y{i+1}' for i in range(n)] + ['lambda^2'] + [f'x1_{i+1}' for i in range(n)] + [f'x2_{i+1}' for i in range(n)]
    df = pd.DataFrame(history, columns=cols)
    print(df.to_string(
        index=False,
        float_format=lambda v: f"{v:.{precision}f}"
    ))

    # Final eigenvalue
    lambda1_final = np.sqrt(eigen2)
    return lambda1_final, x1, x2


## Kết quả

In [6]:
# Define your square matrix A
A = input_matrix('PWDF_input_A.txt', convert_fractions=False)
print("\nMatrix A:"); output_matrix(A)


Matrix A:
 3.0  5.0  3.0 -2.0  4.0
 4.0  1.0 -6.0  0.0  3.0
 2.0  0.0  1.0  7.0 -2.0
 3.0  1.0  4.0  5.0  8.0
-2.0  4.0  0.0  0.0  3.0


## TH1

In [7]:
# Define an initial guess vector x0
A = input_matrix('PWDF_input_A.txt', convert_fractions=False)
x0 = np.array([1, 1, 1, 1, 1])

# Call the power_method function
eigenvalue, eigenvector = power_method(A, x0)

# Display final results
print("Dominant Eigenvalue:", eigenvalue)
print("Corresponding Eigenvector:", eigenvector)

 Iteration              y1              y2              y3               y4              y5          lambda              x1              x2              x3              x4              x5
         1 13.000000000000  2.000000000000  8.000000000000  21.000000000000  5.000000000000 21.000000000000  0.619047619048  0.095238095238  0.380952380952  1.000000000000  0.238095238095
         2  2.428571428571  1.000000000000  8.142857142857  10.380952380952 -0.142857142857 10.380952380952  0.233944954128  0.096330275229  0.784403669725  1.000000000000 -0.013761467890
         3  1.481651376147 -3.715596330275  8.279816513761   8.825688073394 -0.123853211009  8.825688073394  0.167879417879 -0.420997920998  0.938149688150  1.000000000000 -0.014033264033
         4 -0.843035343035 -5.420478170478  8.301975051975   8.722972972973 -2.061850311850  8.722972972973 -0.096645415003 -0.621402609784  0.951736876601  1.000000000000 -0.236370136448
         5 -3.487219209915 -7.427515938748  8.231186319490  

### TH2

In [8]:
A2 = input_matrix('PWDF_input_A2.txt', convert_fractions=False)
x0 = np.array([1, 1, 1], dtype=float)

lambda1, vec1, vec2 = power_method_case2(A2, x0,
                                        tol=1e-8,
                                        max_iter=100,
                                        precision=8)
print("λ1 ≈", lambda1)
print("Eigenvector for λ1:", vec1)
print("Eigenvector for λ2 = -λ1:", vec2)

 Iteration         y1         y2         y3   lambda^2       x1_1        x1_2       x1_3       x2_1       x2_2        x2_3
         1 1.00000000 1.00000000 1.00000000 9.00000000 0.14285714 -0.28571429 1.00000000 0.20000000 0.80000000 -1.00000000
         2 9.00000000 5.00000000 5.00000000 9.00000000 0.19148936 -0.72340426 1.00000000 0.20454545 1.00000000 -0.84090909
λ1 ≈ 3.0
Eigenvector for λ1: [0.19148936170212766 -0.723404255319149 1.0]
Eigenvector for λ2 = -λ1: [0.20454545454545456 1.0 -0.8409090909090909]


# Phương pháp Bậc thang - tìm giá trị riêng trội tiếp theo

## Thuật toán

## Kết quả